<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/makingSmallCorpusBukcet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9e85  GCE       3                                       RUNNING  us-central1-f


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 30 11:40 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '209206846'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
# doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6351288

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls queries_train.json

# adding our python module to the cluster
sc.addFile("/home/dataproc/queries_train.json")
sys.path.insert(0,SparkFiles.getRootDirectory())



queries_train.json


Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
import json
with open("queries_train.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [ ]:
all_docs_id_from_jason = set()
for i in range(len(jsonObject)):
  for doc_id in list(jsonObject.items())[i][1]:
    all_docs_id_from_jason.add(doc_id)
    
print(all_docs_id_from_jason)

{2310146, 970755, 17668101, 964614, 16389, 51759111, 1372169, 321546, 49051658, 54671372, 651278, 5892113, 456722, 16635934, 1380383, 55679006, 28985374, 25935906, 15357987, 19431459, 5144613, 38950, 11147298, 1857574, 5027882, 11028525, 1185840, 48490547, 55836725, 3514423, 6428730, 9254970, 42799166, 55947330, 878659, 2116, 2117, 63809606, 1417287, 1996872, 50602056, 9306179, 1151047, 22423628, 1790029, 33255495, 12789839, 886856, 22736969, 16842834, 9513043, 56107088, 11665493, 37646421, 1321047, 18952271, 43343961, 9988187, 2721889, 2082914, 1390689, 252008, 4253801, 2461806, 489585, 51546226, 42555506, 3762294, 52209782, 1017976, 54046846, 194687, 49016960, 979072, 61603971, 1386629, 2199688, 233609, 43030666, 2578570, 43794572, 30861, 31326350, 19988623, 3639440, 168079, 43794574, 48640150, 188572, 53215263, 17596651, 56494240, 1646753, 41726116, 60739751, 184488, 36827305, 57374888, 10328235, 1511596, 192685, 49029294, 1626287, 2883760, 5222577, 4294832, 42528947, 16615604, 2813

In [ ]:
print(len(all_docs_id_from_jason))

1189


In [ ]:
print(type(parquetFile))

# new_training_bucket = parquetFile.filter(lambda x: x['id'] in all_docs_id_from_jason)
new_training_bucket = parquetFile.filter("id in ({})".format(','.join(map(str, all_docs_id_from_jason))))



# new_training_bucket = parquetFile.rdd.filter(lambda x : x["id"] in all_docs_id_from_jason)

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
new_training_bucket = new_training_bucket.select("id","title","text","anchor_text").distinct()
new_training_bucket.show()
# new_training_bucket.show(10, truncate=True)

+--------+--------------------+--------------------+--------------------+
|      id|               title|                text|         anchor_text|
+--------+--------------------+--------------------+--------------------+
|24724090|    Economy of Milan|thumb|Borsa Itali...|[{43807, Farsley}...|
|24230253|     Sabra (company)|'''Sabra Dipping ...|[{283083, Foggia}...|
|23830729|       Money burning|'''Money burning'...|[{8983183, KCB Sa...|
|43794572|Rick and Morty (s...|The first season ...|[{26091326, Go So...|
|43794574|Rick and Morty (s...|The second season...|[{26091326, State...|
|50191962|Timeline of LinkedIn|alt=LinkedIn logo...|[{970755, LinkedI...|
|50137861|International exp...|'''Netflix''' is ...|[{175537, Netflix...|
|65967176|List of awards an...|The '''Marvel Cin...|[{27306717, Marve...|
|66422422|    D.P. (TV series)|'''''D.P.''''' ()...|[{25584621, Drama...|
|65708464|List of most expe...|Below lists some ...|[{23908541, house...|
|66040084|2022 FIFA World C...|The '''

In [ ]:
# new_training_bucket = new_training_bucket.toDF()
# new_training_bucket.show(10, truncate=True)
new_training_bucket.count()
# new_training_bucket.count()

1189

In [ ]:
index_src = "new_training_bucket2.parquet" #this is writing the new corpus to a bucket in gcp
index_dst = f'gs://{bucket_name}/{index_src}'
new_training_bucket.write.parquet(index_dst)